# Setup

In [1]:
import numpy as np
import os
import glob
import pandas as pd
import sys
import yaml

## Process Config

In [2]:
print('Starting transformation. Working directory: {}'.format(os.getcwd()))

Starting transformation. Working directory: /Users/zhafen/repos/root-dash/src


In [3]:
with open('./config.yml', "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [4]:
input_dir = os.path.join(config['data_dir'], config['input_dirname'])

In [5]:
def get_fp_of_most_recent_file(pattern):
    fps = glob.glob(pattern)
    ind_selected = np.argmax([os.path.getctime(_) for _ in fps])
    return fps[ind_selected]

In [6]:
data_fp = os.path.join(input_dir, config['website_data_file_pattern'])
data_fp = get_fp_of_most_recent_file(data_fp)

In [7]:
press_office_data_fp = os.path.join(input_dir, config['press_office_data_file_pattern'])
press_office_data_fp = get_fp_of_most_recent_file(press_office_data_fp)

In [8]:
output_dir = os.path.join(config['data_dir'], config['output_dirname'])

In [9]:
grouping_labels = [group_by_i.lower().replace(' ', '_') for group_by_i in config['groupings']]

# Transform Website Data

## Preprocessing

In [10]:
# Load data
df = pd.read_csv(data_fp, parse_dates=['Date',])

In [11]:
# Drop drafts
df.drop(df.index[df['Date'].dt.year == 1970], axis='rows', inplace=True)

In [12]:
# Drop weird articles---ancient ones w/o a title or press type
df.dropna(axis='rows', how='any', subset=['Title', 'Press Types',], inplace=True)

In [13]:
# Get rid of HTML ampersands
for str_column in ['Title', 'Research Topics', 'Categories']:
    df[str_column] = df[str_column].str.replace('&amp;', '&')

In [15]:
# Get date bins
start_year = df['Date'].min().year - 1
end_year = df['Date'].max().year + 1
date_bins = pd.date_range(
    '{} {}'.format(config['start_of_year'], start_year),
    pd.Timestamp.now() + pd.offsets.DateOffset(years=1),
    freq = pd.offsets.DateOffset(years=1),
)
date_bin_labels = date_bins.year[:-1]

In [16]:
# Add the year published (using the above start date)
df['Year'] = pd.cut(df['Date'], date_bins, labels=date_bin_labels) 

## Grouped Counts

In [17]:
for i, group_by_i in enumerate(config['groupings']):
    df_i = df.copy()

    # Explode and group
    df_i[group_by_i] = df_i[group_by_i].str.split('|')
    df_i = df_i.explode(group_by_i)

    # Get counts
    counts = df_i.pivot_table(index='Year', columns=group_by_i, values='id', aggfunc='count')
    
    # Save
    output_fn = 'counts.{}.csv'.format(grouping_labels[i])
    counts_output_dir = os.path.join(output_dir, 'counts')
    os.makedirs(counts_output_dir, exist_ok=True)
    output_fp = os.path.join(counts_output_dir, output_fn)
    counts.to_csv(output_fp,)
    print('Saved counts grouped by {} at: {}'.format(group_by_i, output_fp))

Saved counts grouped by Research Topics at: ../data/processed_data/counts/counts.research_topics.csv
Saved counts grouped by Press Types at: ../data/processed_data/counts/counts.press_types.csv
Saved counts grouped by Categories at: ../data/processed_data/counts/counts.categories.csv


# Coordinate Manual Data

## Original Format

In [18]:
# Load press data
press_df = pd.read_excel(press_office_data_fp)
press_df.set_index('id', inplace=True)
if 'Title (optional)' in press_df.columns:
    press_df.drop('Title (optional)', axis='columns', inplace=True)
for column in ['Press Mentions', 'People Reached']:
    press_df[column] = press_df[column].astype('Int64')

In [19]:
# Combine with website data
combined_df = df.set_index('id').join(press_df)

In [20]:
# Save combined data
output_fp = os.path.join(output_dir, config['combined_filename'])
combined_df.to_csv(output_fp)
print('Saved full press data at: {}'.format(output_fp))

Saved full press data at: ../data/processed_data/press.csv


## Exploded Format
(Alternative that parses the categories.)

In [21]:
# Explode and group
exploded_df = combined_df.copy()
for group_by_i in config['groupings']:
    exploded_df[group_by_i] = exploded_df[group_by_i].str.split('|')
    exploded_df = exploded_df.explode(group_by_i)

In [22]:
# Save
base, ext = os.path.splitext(config['combined_filename'])
exploded_filename = '{}.exploded{}'.format(base, ext)
output_fp = os.path.join(output_dir, exploded_filename)
exploded_df.to_csv(output_fp)
print('Saved expanded press data at: {}'.format(output_fp))

Saved expanded press data at: ../data/processed_data/press.exploded.csv
